In [1]:
import functools

import jax
import numpy as np
import scanpy as sc

from cfp.metrics import compute_mean_metrics, compute_metrics, compute_metrics_fast
import cfp.preprocessing as cfpp

In [2]:
split = 4

In [3]:
adata_preds = sc.read(f"/lustre/groups/ml01/workspace/ot_perturbation/models/cpa/combosciplex/adata_with_predictions_{split}.h5ad")

/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [4]:
adata_train_path = f"/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_train_{split}.h5ad"
adata_test_path = f"/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_test_{split}.h5ad"
adata_ood_path = f"/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_ood_{split}.h5ad"

In [5]:
adata_train = sc.read(adata_train_path)
adata_test = sc.read(adata_test_path)
adata_ood = sc.read(adata_ood_path)

In [6]:
adata_ref_test = adata_test[adata_test.obs["condition"]!="control"].copy()
cfpp.centered_pca(adata_ref_test, n_comps=10)

adata_ref_ood = adata_ood[adata_ood.obs["condition"]!="control"].copy()
cfpp.centered_pca(adata_ref_ood, n_comps=10)

In [7]:
adata_pred_test = adata_preds[adata_preds.obs["split"]=="test"]
adata_pred_test.layers["CPA_PRED"] = adata_pred_test.obsm["CPA_pred"]
cfpp.project_pca(query_adata=adata_pred_test, ref_adata=adata_ref_test, layer="CPA_PRED")
cfpp.project_pca(query_adata=adata_test, ref_adata=adata_ref_test)
test_data_target_encoded = {}
test_data_target_decoded = {}
test_data_target_encoded_predicted = {}
test_data_target_decoded_predicted = {}
for cond in adata_test.obs["condition"].cat.categories:
    if cond == "control":
        continue
    test_data_target_encoded[cond] = adata_test[adata_test.obs["condition"] == cond].obsm["X_pca"]
    test_data_target_decoded[cond] = adata_test[adata_test.obs["condition"] == cond].X.toarray()
    test_data_target_decoded_predicted[cond] = adata_pred_test[adata_pred_test.obs["condition"] == cond].obsm["CPA_pred"]
    test_data_target_encoded_predicted[cond] = adata_pred_test[adata_pred_test.obs["condition"] == cond].obsm["X_pca"]

adata_pred_ood = adata_preds[adata_preds.obs["split"]=="ood"]
adata_pred_ood.layers["CPA_PRED"] = adata_pred_ood.obsm["CPA_pred"]
cfpp.project_pca(query_adata=adata_pred_ood, ref_adata=adata_ref_ood, layer="CPA_PRED")
cfpp.project_pca(query_adata=adata_ood, ref_adata=adata_ref_ood)
ood_data_target_encoded = {}
ood_data_target_decoded = {}
ood_data_target_encoded_predicted = {}
ood_data_target_decoded_predicted = {}
for cond in adata_ood.obs["condition"].cat.categories:
    if cond == "control":
        continue
    ood_data_target_encoded[cond] = adata_ood[adata_ood.obs["condition"] == cond].obsm["X_pca"]
    ood_data_target_decoded[cond] = adata_ood[adata_ood.obs["condition"] == cond].X.toarray()
    ood_data_target_decoded_predicted[cond] = adata_pred_ood[adata_pred_ood.obs["condition"] == cond].obsm["CPA_pred"]
    ood_data_target_encoded_predicted[cond] = adata_pred_ood[adata_pred_ood.obs["condition"] == cond].obsm["X_pca"]

/tmp/ipykernel_1796253/2350350615.py:2: ImplicitModificationWarning: Setting element `.layers['CPA_PRED']` of view, initializing view as actual.
  adata_pred_test.layers["CPA_PRED"] = adata_pred_test.obsm["CPA_pred"]
/tmp/ipykernel_1796253/2350350615.py:18: ImplicitModificationWarning: Setting element `.layers['CPA_PRED']` of view, initializing view as actual.
  adata_pred_ood.layers["CPA_PRED"] = adata_pred_ood.obsm["CPA_pred"]


In [8]:
test_deg_dict = {
    k: v
    for k, v in adata_train.uns["rank_genes_groups_cov_all"].items()
    if k in test_data_target_decoded_predicted.keys()
}
ood_deg_dict = {
    k: v
    for k, v in adata_train.uns["rank_genes_groups_cov_all"].items()
    if k in ood_data_target_decoded_predicted.keys()
}

In [9]:
def get_mask(x, y):
    return x[:, [gene in y for gene in adata_train.var_names]]


ood_deg_target_decoded_predicted = jax.tree_util.tree_map(get_mask, ood_data_target_decoded_predicted, ood_deg_dict)
ood_deg_target_decoded = jax.tree_util.tree_map(get_mask, ood_data_target_decoded, ood_deg_dict)

test_deg_target_decoded_predicted = jax.tree_util.tree_map(get_mask, test_data_target_decoded_predicted, test_deg_dict)
test_deg_target_decoded = jax.tree_util.tree_map(get_mask, test_data_target_decoded, test_deg_dict)


In [10]:
deg_ood_metrics = jax.tree_util.tree_map(compute_metrics, ood_deg_target_decoded, ood_deg_target_decoded_predicted)
deg_mean_ood_metrics = compute_mean_metrics(deg_ood_metrics, prefix="deg_ood_")

deg_test_metrics = jax.tree_util.tree_map(compute_metrics, test_deg_target_decoded, test_deg_target_decoded_predicted)
deg_mean_test_metrics = compute_mean_metrics(deg_test_metrics, prefix="deg_test_")


In [11]:
test_metrics_encoded = jax.tree_util.tree_map(
    compute_metrics, test_data_target_encoded, test_data_target_encoded_predicted
)
mean_test_metrics_encoded = compute_mean_metrics(test_metrics_encoded, prefix="encoded_test_")

test_metrics_decoded = jax.tree_util.tree_map(
    compute_metrics_fast, test_data_target_decoded, test_data_target_decoded_predicted
)
mean_test_metrics_decoded = compute_mean_metrics(test_metrics_decoded, prefix="decoded_test_")

In [12]:
ood_metrics_encoded = jax.tree_util.tree_map(
    compute_metrics, ood_data_target_encoded, ood_data_target_encoded_predicted
)
mean_ood_metrics_encoded = compute_mean_metrics(ood_metrics_encoded, prefix="encoded_ood_")

ood_metrics_decoded = jax.tree_util.tree_map(
    compute_metrics, ood_data_target_decoded, ood_data_target_decoded_predicted
)
mean_ood_metrics_decoded = compute_mean_metrics(ood_metrics_decoded, prefix="decoded_ood_")

In [13]:
mean_ood_metrics_decoded

{'decoded_ood_r_squared': 0.9609271883964539,
 'decoded_ood_sinkhorn_div_1': 80.21781158447266,
 'decoded_ood_sinkhorn_div_10': 47.63763936360677,
 'decoded_ood_sinkhorn_div_100': 3.779438018798828,
 'decoded_ood_e_distance': 6.959160346564683,
 'decoded_ood_mmd': 0.1933374578754107}

In [14]:
mean_test_metrics_decoded

{'decoded_test_r_squared': 0.7165579080581665,
 'decoded_test_e_distance': 55.682782278740014,
 'decoded_test_mmd_distance': 0.24415573060512544}

In [15]:
mean_ood_metrics_decoded

{'decoded_ood_r_squared': 0.9609271883964539,
 'decoded_ood_sinkhorn_div_1': 80.21781158447266,
 'decoded_ood_sinkhorn_div_10': 47.63763936360677,
 'decoded_ood_sinkhorn_div_100': 3.779438018798828,
 'decoded_ood_e_distance': 6.959160346564683,
 'decoded_ood_mmd': 0.1933374578754107}

In [16]:
mean_ood_metrics_encoded

{'encoded_ood_r_squared': -0.6885734895865122,
 'encoded_ood_sinkhorn_div_1': 7.413774887720744,
 'encoded_ood_sinkhorn_div_10': 3.842919667561849,
 'encoded_ood_sinkhorn_div_100': 2.9025758107503257,
 'encoded_ood_e_distance': 5.595856366422133,
 'encoded_ood_mmd': 0.11353601018587749}

In [17]:
deg_mean_ood_metrics

{'deg_ood_r_squared': 0.9407360255718231,
 'deg_ood_sinkhorn_div_1': 8.015044371287027,
 'deg_ood_sinkhorn_div_10': 2.1661787033081055,
 'deg_ood_sinkhorn_div_100': 1.8562882741292317,
 'deg_ood_e_distance': 3.6483341832598826,
 'deg_ood_mmd': 0.24738523860772452}

In [18]:
deg_mean_test_metrics

{'deg_test_r_squared': 0.5044333672523499,
 'deg_test_sinkhorn_div_1': 25.997939863204955,
 'deg_test_sinkhorn_div_10': 17.187143688201903,
 'deg_test_sinkhorn_div_100': 16.001890201568603,
 'deg_test_e_distance': 31.83817461787437,
 'deg_test_mmd': 0.31954491019248965}

In [19]:
mean_test_metrics_decoded

{'decoded_test_r_squared': 0.7165579080581665,
 'decoded_test_e_distance': 55.682782278740014,
 'decoded_test_mmd_distance': 0.24415573060512544}

In [20]:
mean_ood_metrics_decoded

{'decoded_ood_r_squared': 0.9609271883964539,
 'decoded_ood_sinkhorn_div_1': 80.21781158447266,
 'decoded_ood_sinkhorn_div_10': 47.63763936360677,
 'decoded_ood_sinkhorn_div_100': 3.779438018798828,
 'decoded_ood_e_distance': 6.959160346564683,
 'decoded_ood_mmd': 0.1933374578754107}

In [21]:
deg_mean_ood_metrics

{'deg_ood_r_squared': 0.9407360255718231,
 'deg_ood_sinkhorn_div_1': 8.015044371287027,
 'deg_ood_sinkhorn_div_10': 2.1661787033081055,
 'deg_ood_sinkhorn_div_100': 1.8562882741292317,
 'deg_ood_e_distance': 3.6483341832598826,
 'deg_ood_mmd': 0.24738523860772452}

In [22]:
mean_test_metrics_decoded

{'decoded_test_r_squared': 0.7165579080581665,
 'decoded_test_e_distance': 55.682782278740014,
 'decoded_test_mmd_distance': 0.24415573060512544}

In [23]:
mean_test_metrics_encoded

{'encoded_test_r_squared': -1.2700761236426246,
 'encoded_test_sinkhorn_div_1': 31.080936393737794,
 'encoded_test_sinkhorn_div_10': 27.395958175659178,
 'encoded_test_sinkhorn_div_100': 26.197151412963866,
 'encoded_test_e_distance': 52.12622266519961,
 'encoded_test_mmd': 0.2468197053670883}

In [24]:
ood_metrics_encoded

{'Cediranib+PCI-34051': {'r_squared': 0.49938827753067017,
  'sinkhorn_div_1': 5.90039587020874,
  'sinkhorn_div_10': 2.5057907104492188,
  'sinkhorn_div_100': 1.6214227676391602,
  'e_distance': 3.038547392128918,
  'mmd': 0.101293705},
 'Givinostat+Crizotinib': {'r_squared': -1.6983377933502197,
  'sinkhorn_div_1': 6.649031639099121,
  'sinkhorn_div_10': 2.9800233840942383,
  'sinkhorn_div_100': 1.9597272872924805,
  'e_distance': 3.6989421701647203,
  'mmd': 0.10432935},
 'Givinostat+SRT2104': {'r_squared': -1.2091290950775146,
  'sinkhorn_div_1': 6.1728363037109375,
  'sinkhorn_div_10': 2.4377031326293945,
  'sinkhorn_div_100': 1.4428091049194336,
  'e_distance': 2.6638451504390206,
  'mmd': 0.099010326},
 'Givinostat+Sorafenib': {'r_squared': -1.042428731918335,
  'sinkhorn_div_1': 14.048883438110352,
  'sinkhorn_div_10': 10.8548583984375,
  'sinkhorn_div_100': 10.13344955444336,
  'e_distance': 20.11791052856899,
  'mmd': 0.18281622},
 'control+Givinostat': {'r_squared': -1.38003

In [25]:
output_dir = "/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/results/cpa"

In [26]:
import os
import pandas as pd

pd.DataFrame.from_dict(ood_metrics_encoded).to_csv(os.path.join(output_dir, f"ood_metrics_encoded_{split}.csv"))
pd.DataFrame.from_dict(ood_metrics_decoded).to_csv(os.path.join(output_dir, f"ood_metrics_decoded_{split}.csv"))
pd.DataFrame.from_dict(test_metrics_encoded).to_csv(os.path.join(output_dir, f"test_metrics_encoded_{split}.csv"))
pd.DataFrame.from_dict(test_metrics_decoded).to_csv(os.path.join(output_dir, f"test_metrics_decoded_{split}.csv"))
pd.DataFrame.from_dict(deg_test_metrics).to_csv(os.path.join(output_dir, f"test_metrics_deg_{split}.csv"))
pd.DataFrame.from_dict(deg_ood_metrics).to_csv(os.path.join(output_dir, f"ood_metrics_ood_{split}.csv"))
